In [1]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

In [2]:
docs = [
    Document(
        page_content="一群科学家利用琥珀中蚊子体内的恐龙血液克隆出了恐龙，并建立了一个名为‘侏罗纪公园’的野生动物园。然而，在一次预览参观中，系统意外瘫痪，各种凶猛的恐龙挣脱束缚，在岛上爆发了毁灭性的混乱，人们必须在恐龙的追杀中寻找生存的希望。",
        metadata={"year": 1993, "rating": 7.7, "genre": "科幻小说"},
    ),
    Document(
        page_content="故事发生在1920年北洋年间中国南方，马邦德花钱买官，购得“萨南康省”的县长一职，坐“马拉的火车”赴任途中遭马匪张麻子劫持。为了保命，马邦德谎称自己是师爷，随张麻子走马上任鹅城。在那儿，他们与当地恶霸黄四郎展开了一场关于权力、金钱与正义的斗智斗勇。",
        metadata={"year": 2010, "director": "姜文", "rating": 8.2},
    ),
    Document(
        page_content="话说孙悟空护送唐三藏前往西天取经，半路却和牛魔王合谋要杀害唐三藏，并偷走了紫霞仙子持有的月光宝盒。观音闻讯赶到将孙悟空收服，为了给他改过自新的机会，让他五百年后投胎为强盗头目至尊宝。至尊宝在寻找白晶晶的过程中意外发现月光宝盒，并穿越回五百年前，与紫霞仙子展开了一段跨越时空的凄美爱情。",
        metadata={"year": 1994, "director": "刘镇伟", "rating": 8.6},
    ),
    Document(
        page_content="故事背景设定在2075年，讲述了太阳即将毁灭，毁灭之后的太阳系已经不适合人类生存。面对绝境，人类开启了“流浪地球”计划，试图带着地球逃离太阳系去往新家园。在经过木星轨道时，由于引力激增，地球面临坠入木星的危机，全球救援队展开了一场争分夺秒的生死营救。",
        metadata={"year": 2019, "director": "郭帆", "rating": 8.3},
    ),
    Document(
        page_content="该片讲述了耿浩在遭遇婚姻破裂的打击后，陷入极度消沉。他的好哥们郝义为了带他散心，开启了一场通往大理的荒诞而有趣的“寻爱之旅”。该片采用双线叙事的手法，以耿浩现在的自愈之旅和康小雨当年的进藏之旅交织，最终揭示了两人关系的起点与终点，传达了放下过去、重新出发的主题。",
        metadata={"year": 2014, "genre": "喜剧"},
    ),
]

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(
  base_url="http://127.0.0.1:1234/v1",
  openai_api_key="lm-studio",
  model="text-embedding-bge-large-zh-v1.5",
  check_embedding_ctx_length=False
)

vectorstore=Chroma.from_documents(docs,embeddings)

In [4]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description='电影的类型。["科幻小说", "喜剧", "剧情片", "惊悚片", "爱情片", "动作片", "动画片"]之一',
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="电影上映年份",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="电影导演的名字",
        type="string",
    ),
    AttributeInfo(
        name="rating", 
        description="电影评分为 1-10", 
        type="float"
    ),
]

In [5]:
document_content_description="电影的简要概述"

In [6]:
from langchain_openai import OpenAI,ChatOpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.3
)

In [8]:
retriever=SelfQueryRetriever.from_llm(
    model,
    vectorstore,
    document_content_description,   #是对文档内容的简短描述（字符串）。它告诉 LLM 数据库里存的是什么
    metadata_field_info,
    enable_limit=True,      #允许 LLM 识别“数量限制”
    search_kwargs={"k": 2}
)

In [9]:
retriever.invoke("给我推荐一部评分8.5以上的电影")

[Document(metadata={'director': '刘镇伟', 'rating': 8.6, 'year': 1994}, page_content='话说孙悟空护送唐三藏前往西天取经，半路却和牛魔王合谋要杀害唐三藏，并偷走了紫霞仙子持有的月光宝盒。观音闻讯赶到将孙悟空收服，为了给他改过自新的机会，让他五百年后投胎为强盗头目至尊宝。至尊宝在寻找白晶晶的过程中意外发现月光宝盒，并穿越回五百年前，与紫霞仙子展开了一段跨越时空的凄美爱情。')]

In [10]:
retriever.invoke("给我推荐一部寻爱之旅的电影")

[Document(metadata={'year': 2014, 'genre': '喜剧'}, page_content='该片讲述了耿浩在遭遇婚姻破裂的打击后，陷入极度消沉。他的好哥们郝义为了带他散心，开启了一场通往大理的荒诞而有趣的“寻爱之旅”。该片采用双线叙事的手法，以耿浩现在的自愈之旅和康小雨当年的进藏之旅交织，最终揭示了两人关系的起点与终点，传达了放下过去、重新出发的主题。'),
 Document(metadata={'year': 1994, 'rating': 8.6, 'director': '刘镇伟'}, page_content='话说孙悟空护送唐三藏前往西天取经，半路却和牛魔王合谋要杀害唐三藏，并偷走了紫霞仙子持有的月光宝盒。观音闻讯赶到将孙悟空收服，为了给他改过自新的机会，让他五百年后投胎为强盗头目至尊宝。至尊宝在寻找白晶晶的过程中意外发现月光宝盒，并穿越回五百年前，与紫霞仙子展开了一段跨越时空的凄美爱情。')]